In [9]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(100_000_000_000)

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics

import shared_utils
import branca
from utils import *

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis as rt
import importlib

In [10]:
%%sql -o rt_coverage

SELECT calitp_itp_id, calitp_url_number, MIN(vehicle.timestamp) as min_timestamp, MAX(vehicle.timestamp) as max_timestamp
FROM `cal-itp-data-infra.gtfs_rt.vehicle_positions`
GROUP BY calitp_itp_id, calitp_url_number
ORDER BY max_timestamp DESC

,calitp_itp_id,calitp_url_number,min_timestamp,max_timestamp
0,360,0,1644017957,1644537593
1,45,0,1420441687,1644537592
2,284,0,1644191947,1644537592
3,301,0,1644192006,1644537591
4,246,1,1644192008,1644537591
5,346,0,1644122957,1644537591
6,110,1,1644234376,1644537590
7,200,0,1644191970,1644537590
8,194,0,1644160780,1644537590
9,208,0,1644191948,1644537590


In [11]:
## set system time
os.environ['TZ'] = 'America/Los_Angeles'
time.tzset()
time.tzname

('PST', 'PDT')

In [12]:
rt_coverage.max_timestamp = rt_coverage.max_timestamp.apply(convert_ts)
rt_coverage.min_timestamp = rt_coverage.min_timestamp.apply(convert_ts)

In [14]:
rt_coverage

,calitp_itp_id,calitp_url_number,min_timestamp,max_timestamp
0,360,0,2022-02-04 15:39:17,2022-02-10 15:59:53
1,45,0,2015-01-04 23:08:07,2022-02-10 15:59:52
2,284,0,2022-02-06 15:59:07,2022-02-10 15:59:52
3,301,0,2022-02-06 16:00:06,2022-02-10 15:59:51
4,246,1,2022-02-06 16:00:08,2022-02-10 15:59:51
5,346,0,2022-02-05 20:49:17,2022-02-10 15:59:51
6,110,1,2022-02-07 03:46:16,2022-02-10 15:59:50
7,200,0,2022-02-06 15:59:30,2022-02-10 15:59:50
8,194,0,2022-02-06 07:19:40,2022-02-10 15:59:50
9,208,0,2022-02-06 15:59:08,2022-02-10 15:59:50


https://airtable.com/shrXjIHW4f4Bk2PSG

In [15]:
air_view = pd.read_csv('airtable_rt_status.csv').rename(columns={'ITP ID': 'calitp_itp_id', 'GTFS Realtime Status': 'rt_status'})
air_view.calitp_itp_id = air_view.calitp_itp_id.fillna(9999).astype(int)

In [16]:
air_view.columns

Index(['Name', 'calitp_itp_id', 'Reporting Category', 'Caltrans District',
       'GTFS Static Status', 'rt_status', 'Missing Static',
       'Services Needing Alerts',
       'Services Needing TripUpdates or VehiclePositions'],
      dtype='object')

In [17]:
air_joined = air_view >> inner_join(_, rt_coverage, on = 'calitp_itp_id') >> distinct(_.calitp_itp_id, _keep_all=True)

In [18]:
air_joined.to_parquet('airtable_joined.parquet')

In [19]:
air_joined.to_csv('airtable_joined.csv')

In [20]:
air_joined.shape

(48, 12)

In [21]:
air_joined

,Name,calitp_itp_id,Reporting Category,Caltrans District,GTFS Static Status,rt_status,Missing Static,Services Needing Alerts,Services Needing TripUpdates or VehiclePositions,calitp_url_number,min_timestamp,max_timestamp
0,Ventura County Transportation Commission,380,Core,07 - Los Angeles,Static OK,RT OK,NaN,Valley Express Dial-A-Ride,Valley Express Dial-A-Ride,0,2022-02-03 20:07:26,2022-02-10 15:59:49
1,City of Santa Monica,300,Core,07 - Los Angeles,Static OK,RT OK,NaN,"Mobility on Demand Everyday, Santa Monica Coll...","Mobility on Demand Everyday, Santa Monica Coll...",0,2022-02-06 15:55:20,2022-02-10 15:59:35
2,Los Angeles County Metropolitan Transportation...,182,Core,07 - Los Angeles,Static OK,RT OK,NaN,LA Metro Micro,LA Metro Micro,1,2022-02-06 15:58:17,2022-02-10 15:59:45
3,City of Pasadena,243,Core,07 - Los Angeles,Static OK,RT OK,NaN,Pasadena Dial-A-Ride,Pasadena Dial-A-Ride,0,2022-02-06 15:57:19,2022-02-10 15:59:25
4,City of Santa Clarita,295,Core,07 - Los Angeles,Static OK,RT OK,NaN,"Go! Santa Clarita, Santa Clarita Summer Trolly...","Go! Santa Clarita, Santa Clarita Summer Trolly...",0,2022-02-06 15:59:20,2022-02-09 17:34:57
5,Long Beach Transit,170,Core,07 - Los Angeles,Static Incomplete,RT Incomplete,"AquaLink, UCLA/Westwood Commuter Express","Galaxy Express, AquaLink, UCLA/Westwood Commut...","Galaxy Express, AquaLink, UCLA/Westwood Commut...",0,2022-02-06 15:54:38,2022-02-10 15:59:16
6,City of Burbank,45,Core,07 - Los Angeles,Static OK,RT OK,NaN,Burbank Bus Senior and Disabled,Burbank Bus Senior and Disabled,0,2015-01-04 23:08:07,2022-02-10 15:59:52
7,City of Commerce,75,Core,07 - Los Angeles,Static OK,RT OK,NaN,Commerce Dial-A-Ride,Commerce Dial-A-Ride,0,2022-02-06 14:46:27,2022-02-10 15:59:49
8,Alameda-Contra Costa Transit District,4,Core,04 - Oakland,Static OK,RT OK,NaN,East Bay Paratransit,East Bay Paratransit,1,2022-02-06 15:59:47,2022-02-10 15:59:34
9,Solano County Transit,310,Core,04 - Oakland,Static OK,RT OK,NaN,SolTrans ADA Paratransit,SolTrans ADA Paratransit,1,2022-02-06 16:00:00,2022-02-10 15:59:30


In [22]:
rt_coverage >> inner_join(_, tbl.gtfs_schedule.agency() >> collect(), on = ['calitp_itp_id']) >> filter(_.calitp_itp_id == 290)

/opt/conda/lib/python3.9/site-packages/siuba/sql/utils.py:52: SAWarning: Dialect bigquery:bigquery will not make use of SQL compilation caching as it does not set the 'supports_statement_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Dialect maintainers should seek to set this attribute to True after appropriate development and testing for SQLAlchemy 1.4 caching support.   Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/14/cprf)


,calitp_itp_id,calitp_url_number_x,min_timestamp,max_timestamp,calitp_url_number_y,agency_id,agency_name,agency_url,agency_timezone,agency_lang,agency_phone,agency_fare_url,agency_email,calitp_extracted_at
118,290,1,2022-02-06 16:00:01,2022-02-10 15:59:05,1,SM,SamTrans,http://www.samtrans.com,America/Los_Angeles,en,800-660-4287,None,None,2022-01-31
119,290,1,2022-02-06 16:00:01,2022-02-10 15:59:05,0,samtrans-ca-us,SamTrans,http://www.samtrans.com,America/Los_Angeles,en,800-660-4287,None,None,2022-02-09


In [23]:
rt_coverage >> inner_join(_, tbl.gtfs_schedule.agency() >> collect(), on = ['calitp_itp_id']) >> filter(_.calitp_itp_id == 182)

,calitp_itp_id,calitp_url_number_x,min_timestamp,max_timestamp,calitp_url_number_y,agency_id,agency_name,agency_url,agency_timezone,agency_lang,agency_phone,agency_fare_url,agency_email,calitp_extracted_at
84,182,1,2022-02-06 15:58:17,2022-02-10 15:59:45,0,LACMTA,Metro - Los Angeles,https://www.metro.net,America/Los_Angeles,en,(323)466-3876,None,None,2022-02-04
85,182,1,2022-02-06 15:58:17,2022-02-10 15:59:45,1,LACMTA_Rail,Metro - Los Angeles,https://www.metro.net,America/Los_Angeles,en,(323) 466-3876,None,None,2022-02-10
86,182,0,2022-02-06 15:50:22,2022-02-10 15:59:41,0,LACMTA,Metro - Los Angeles,https://www.metro.net,America/Los_Angeles,en,(323)466-3876,None,None,2022-02-04
87,182,0,2022-02-06 15:50:22,2022-02-10 15:59:41,1,LACMTA_Rail,Metro - Los Angeles,https://www.metro.net,America/Los_Angeles,en,(323) 466-3876,None,None,2022-02-10
